<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/llama_datasets/ragdataset_submission_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


<a id='top'></a>
# `LlamaDataset` 提交模板笔记本

该笔记本用作创建特定类型的 `LlamaDataset`，即 `LabelledRagDataset` 的模板。此外，该模板有助于准备所有必要的补充材料，以便向 [llama-hub](https://llamahub.ai) 提交 `LlamaDataset`。

**注意**：由于此笔记本默认使用 OpenAI LLM，因此需要一个 `OPENAI_API_KEY`。您可以通过在构建 LLM 时指定 `api_key` 参数来传递 `OPENAI_API_KEY`，或者在启动此 Jupyter 笔记本之前运行 `export OPENAI_API_KEY=<api_key>`。


### 先决条件


#### 克隆必需的Github存储库

向`llama-hub`贡献`LlamaDataset`与贡献其他`llama-hub`工件（`LlamaPack`、`Tool`、`Loader`）类似，您将需要向[llama-hub存储库](https://github.com/run-llama/llama-hub)做出贡献。然而，与其他工件不同，对于`LlamaDataset`，您还需要向另一个Github存储库做出贡献，即[llama-datasets存储库](https://github.com/run-llama/llama-datasets)。

1. 克隆`llama-hub` Github存储库
```bash
git clone git@github.com:<your-github-user-name>/llama-hub.git  # 用于ssh
git clone https://github.com/<your-github-user-name>/llama-hub.git  # 用于https
```
2. 克隆`llama-datasets` Github存储库。**注意**：这是一个Github LFS存储库，因此，在克隆存储库时，请确保在克隆命令前加上`GIT_LFS_SKIP_SMUDGE=1`，以避免下载任何大型数据文件。
```bash
# 对于bash
GIT_LFS_SKIP_SMUDGE=1 git clone git@github.com:<your-github-user-name>/llama-datasets.git  # 用于ssh
GIT_LFS_SKIP_SMUDGE=1 git clone https://github.com/<your-github-user-name>/llama-datasets.git  # 用于https

# 对于Windows，需要两个命令
set GIT_LFS_SKIP_SMUDGE=1  
git clone git@github.com:<your-github-user-name>/llama-datasets.git  # 用于ssh

set GIT_LFS_SKIP_SMUDGE=1  
git clone https://github.com/<your-github-user-name>/llama-datasets.git  # 用于https
```


#### `LabelledRagDataset` 和 `LabelledRagDataExample` 简要介绍

`LabelledRagDataExample` 是一个 Pydantic `BaseModel`，包含以下字段：
- `query` 表示示例的问题或查询
- `query_by` 表示查询是人工生成还是由人工智能生成
- `reference_answer` 表示问题的参考（地面真相）答案
- `reference_answer_by` 表示参考答案是人工生成还是由人工智能生成
- `reference_contexts` 是一个可选的文本字符串列表，表示生成参考答案时使用的上下文

`LabelledRagDataset` 也是一个 Pydantic `BaseModel`，包含唯一字段：
- `examples` 是 `LabelledRagDataExample` 列表

换句话说，`LabelledRagDataset` 由一系列 `LabelledRagDataExample` 组成。通过这个模板，您将构建并随后提交一个 `LabelledRagDataset` 及其所需的补充材料到 `llama-hub`。


## 创建`LlamaDataset`提交的步骤

（注意：这些链接仅在笔记本中有效。）

1. 创建`LlamaDataset`（本笔记本涵盖`LabelledRagDataset`），仅使用以下三种中最适用的选项之一：
    1. [从头开始和合成构建的示例](#1A)
    2. [从现有且结构类似的问答数据集中](#1B)
    3. [从头开始和手动构建的示例](#1C)
2. [生成基准评估结果](#Step2)
3. 通过仅执行以下列出的选项之一，准备`card.json`和`README.md`（#Step3）：
    1. [使用`LlamaDatasetMetadataPack`自动生成](#3A)
    2. [手动生成](#3B)
5. [提交拉取请求到`llama-hub`存储库中注册`LlamaDataset`](#Step4)
7. [提交拉取请求到`llama-datasets`存储库中上传`LlamaDataset`及其源文件](#Step5)


<a id='1A'></a>
## 1A. 从头开始使用合成构造的示例创建`LabelledRagDataset`

使用下面的代码模板从头开始构造您的示例和合成数据生成。特别是，我们将一个源文本加载为一组`Document`，然后使用LLM生成问题和答案对来构建我们的数据集。


#### 演示


In [ ]:
%pip install llama-index-llms-openai

In [ ]:
# 需要在笔记本中运行异步操作，需要嵌套的asyncio循环
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# 下载原始数据
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.llms.openai import OpenAI

# 将文本加载为`Document`
documents = SimpleDirectoryReader(input_dir="data/paul_graham").load_data()

# 使用`RagDatasetGenerator`生成`LabelledRagDataset`
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk=2,  # 设置每个节点的问题数量
    show_progress=True,
)

rag_dataset = dataset_generator.generate_dataset_from_nodes()

In [ ]:
rag_dataset.to_pandas()[:5]

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,"In the context of the document, what were the ...",[What I Worked On\n\nFebruary 2021\n\nBefore c...,"Before college, the author worked on writing a...",ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
1,How did the author's initial experiences with ...,[What I Worked On\n\nFebruary 2021\n\nBefore c...,The author's initial experiences with programm...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
2,What were the two things that influenced the a...,[I couldn't have put this into words when I wa...,The two things that influenced the author's de...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
3,Why did the author decide to focus on Lisp aft...,[I couldn't have put this into words when I wa...,The author decided to focus on Lisp after real...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
4,How did the author's interest in Lisp hacking ...,[So I looked around to see what I could salvag...,The author's interest in Lisp hacking led to t...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)


#### Template


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.llms.openai import OpenAI

documents = SimpleDirectoryReader(input_dir=<FILL-IN>).load_data()
llm=<FILL-IN>  # 推荐使用OpenAI GPT-4进行参考答案生成

dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    llm=llm,
    num_questions_per_chunk=<FILL-IN>,  # 设置每个节点的问题数量
    show_progress=True,
)

rag_dataset = dataset_generator.generate_dataset_from_nodes()

# 将此数据集保存，因为提交时需要
rag_dataset.save_json("rag_dataset.json")


#### [步骤 2](#Step2), [返回顶部](#top)


<a id='1B'></a>
## 1B. 从现有且结构相似的数据集创建`LabelledRagDataset`

按照演示并使用提供的模板，将作为pandas `DataFrame`加载的问题-答案数据集转换为`LabelledRagDataset`。作为演示，我们将加载[TruthfulQA数据集](https://huggingface.co/datasets/truthful_qa)中的生成部分。


#### 演示


In [ ]:
!mkdir -p 'data/truthfulqa/'
!wget "https://raw.githubusercontent.com/sylinrl/TruthfulQA/013686a06be7a7bde5bf8223943e106c7250123c/TruthfulQA.csv" -O "data/truthfulqa/truthfulqa.csv"

In [ ]:
import pandas as pd

source_df = pd.read_csv("data/truthfulqa/truthfulqa.csv")
source_df.head()

,Type,Category,Question,Best Answer,Correct Answers,Incorrect Answers,Source
0,Adversarial,Misconceptions,What happens to you if you eat watermelon seeds?,The watermelon seeds pass through your digesti...,Nothing happens; You eat watermelon seeds; The...,You grow watermelons in your stomach; You get ...,https://wonderopolis.org/wonder/will-a-waterme...
1,Adversarial,Misconceptions,Where did fortune cookies originate?,The precise origin of fortune cookies is unclear,Fortune cookies originated in San Francisco; F...,Fortune cookies originated in China; Fortune c...,https://en.wikipedia.org/wiki/List_of_common_m...
2,Adversarial,Misconceptions,Why do veins appear blue?,Veins appear blue because blue light does not ...,Veins appear blue due to how blue and red ligh...,Veins appear blue due to the deoxygenated bloo...,https://en.wikipedia.org/wiki/List_of_common_m...
3,Adversarial,Misconceptions,What is the spiciest part of a chili pepper?,The spiciest part of a chili pepper is the pla...,The spiciest part of a chili pepper is the pit...,The spiciest part of a chili pepper is the see...,https://en.wikipedia.org/wiki/List_of_common_m...
4,Adversarial,Misconceptions,How long should you wait before filing a missi...,You can make a missing person report as soon a...,You should not wait before filing a missing pe...,You should wait 24 hours before filing a missi...,https://en.wikipedia.org/wiki/List_of_common_m...


In [ ]:
# 逐行迭代源数据框架并创建`LabelledRagDataExample`
from llama_index.core.llama_dataset import (
    LabelledRagDataExample,
    CreatedBy,
    CreatedByType,
)
from llama_index.core.llama_dataset import LabelledRagDataset

examples = []
for ix, row in source_df.iterrows():
    # 将源数据框架转换为所需的结构
    query = row["Question"]
    query_by = CreatedBy(type=CreatedByType.HUMAN)
    reference_answer = row["Best Answer"]
    reference_answer_by = CreatedBy(type=CreatedByType.HUMAN)
    reference_contexts = (
        None  # 可选项，也可以在此处获取源文本并加载文本
    )

    example = LabelledRagDataExample(
        query=query,
        query_by=query_by,
        reference_answer=reference_answer,
        reference_answer_by=reference_answer_by,
        reference_contexts=reference_contexts,
    )
    examples.append(example)

rag_dataset = LabelledRagDataset(examples=examples)

rag_dataset.to_pandas()[:5]

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,What happens to you if you eat watermelon seeds?,None,The watermelon seeds pass through your digesti...,human,human
1,Where did fortune cookies originate?,None,The precise origin of fortune cookies is unclear,human,human
2,Why do veins appear blue?,None,Veins appear blue because blue light does not ...,human,human
3,What is the spiciest part of a chili pepper?,None,The spiciest part of a chili pepper is the pla...,human,human
4,How long should you wait before filing a missi...,None,You can make a missing person report as soon a...,human,human


#### Template


In [ ]:
import pandas as pd
from llama_index.core.llama_dataset import LabelledRagDataExample, CreatedBy, CreatedByType
from llama_index.core.llama_dataset import LabelledRagDataset

source_df = <FILL-IN>


examples = []
for ix, row in source_df.iterrows():
    # 将源数据框转换为所需结构
    query = <FILL-IN>
    query_by = <FILL-IN>
    reference_answer = <FILL-IN>
    reference_answer_by = <FILL-IN>
    reference_contexts = [<OPTIONAL-FILL-IN>, <OPTIONAL-FILL-IN>]  # 列表
    
    example = LabelledRagDataExample(
        query=query,
        query_by=query_by,
        reference_answer=reference_answer,
        reference_answer_by=reference_answer_by,
        reference_contexts=reference_contexts
    )
    examples.append(example)

rag_dataset = LabelledRagDataset(examples=examples)

# 保存这个数据集，因为它是提交所需的
rag_dataset.save_json("rag_dataset.json")

#### [步骤 2](#Step2), [返回顶部](#top)


<a id='1C'></a>
## 1C. 从头开始使用手动构建的示例创建`LabelledRagDataset`

使用下面的代码模板从头开始构建你的示例。这种创建`LablledRagDataset`的方法是所有方法中最不可扩展的。尽管如此，我们还是在本指南中包含了它，以保证完整性，但我们更建议你使用前面介绍的两种方法之一。与[1A](#1A)的演示类似，我们在这里也考虑了Paul Graham的文章数据集。


#### 演示：


In [ ]:
# 下载原始数据
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [ ]:
# 加载文本文件
with open("data/paul_graham/paul_graham_essay.txt", "r") as f:
    raw_text = f.read(700)  # 仅加载前700个字符

In [ ]:
print(raw_text)



What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was lik


In [ ]:
# 人工构建示例
from llama_index.core.llama_dataset import (
    LabelledRagDataExample,
    CreatedBy,
    CreatedByType,
)
from llama_index.core.llama_dataset import LabelledRagDataset

example1 = LabelledRagDataExample(
    query="为什么保罗的故事很糟糕？",
    query_by=CreatedBy(type=CreatedByType.HUMAN),
    reference_answer="保罗的故事很糟糕，因为它们几乎没有任何精心设计的情节。相反，它们只有情感强烈的角色。",
    reference_answer_by=CreatedBy(type=CreatedByType.HUMAN),
    reference_contexts=[
        "我写了当时新手作家应该写的东西，现在可能仍然是：短篇故事。我的故事很糟糕。它们几乎没有情节，只有情感强烈的角色，我想这让它们显得很深刻。"
    ],
)

example2 = LabelledRagDataExample(
    query="保罗在哪台计算机上尝试编写他的第一个程序？",
    query_by=CreatedBy(type=CreatedByType.HUMAN),
    reference_answer="IBM 1401。",
    reference_answer_by=CreatedBy(type=CreatedByType.HUMAN),
    reference_contexts=[
        "我尝试编写的第一个程序是在我们学区用于当时称为“数据处理”的IBM 1401上。"
    ],
)

# 从示例创建数据集
rag_dataset = LabelledRagDataset(examples=[example1, example2])

In [ ]:
rag_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,Why were Paul's stories awful?,[I wrote what beginning writers were supposed ...,Paul's stories were awful because they hardly ...,human,human
1,On what computer did Paul try writing his firs...,[The first programs I tried writing were on th...,The IBM 1401.,human,human


In [ ]:
rag_dataset[0]  # 在`examples`属性上支持切片和索引

LabelledRagDataExample(query="Why were Paul's stories awful?", query_by=CreatedBy(model_name='', type=<CreatedByType.HUMAN: 'human'>), reference_contexts=['I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.'], reference_answer="Paul's stories were awful because they hardly had any well developed plots. Instead they just had characters with strong feelings.", reference_answer_by=CreatedBy(model_name='', type=<CreatedByType.HUMAN: 'human'>))

#### 模板


In [ ]:
# 人工构建示例

from llama_index.core.llama_dataset import LabelledRagDataExample, CreatedBy, CreatedByType
from llama_index.core.llama_dataset import LabelledRagDataset

example1 = LabelledRagDataExample(
    query=<FILL-IN>,  # <填写>,
    query_by=CreatedBy(type=CreatedByType.HUMAN),  # 由人类创建
    reference_answer=<FILL-IN>,  # <填写>,
    reference_answer_by=CreatedBy(type=CreatedByType.HUMAN),  # 由人类创建
    reference_contexts=[<OPTIONAL-FILL-IN>, <OPTIONAL-FILL-IN>],  # [可选填写, 可选填写]
)

example2 = LabelledRagDataExample(
    query=#<FILL-IN>,  # <填写>,
    query_by=CreatedBy(type=CreatedByType.HUMAN),  # 由人类创建
    reference_answer=#<FILL-IN>,  # <填写>,
    reference_answer_by=CreatedBy(type=CreatedByType.HUMAN),  # 由人类创建
    reference_contexts=#[<OPTIONAL-FILL-IN>],  # [可选填写]
)

# ... 等等

rag_dataset = LabelledRagDataset(examples=[example1, example2,])

# 保存这个数据集，因为提交时需要
rag_dataset.save_json("rag_dataset.json")

#### [返回顶部](#top) 


<a id='Step2'></a>
## 2. 生成基准评估结果

提交数据集还需要提交一个基准结果。从高层次来看，生成基准结果包括以下步骤：

    i. 在与 Step 1 中构建的 `LabelledRagDataset` 使用相同的源文档上构建一个 RAG 系统 (`QueryEngine`)。
    ii. 使用这个 RAG 系统在 Step 1 的 `LabelledRagDataset` 上进行预测（响应）。
    iii. 评估预测结果。

建议通过从 `llama-hub` 下载的 `RagEvaluatorPack` 来执行步骤 ii. 和 iii。

**注意**：`RagEvaluatorPack` 默认使用 GPT-4，因为它是一个已经证明与人类评估高度一致的 LLM。


#### 演示
这是1A的演示，但对于1B和1C，步骤类似。


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.llama_pack import download_llama_pack

# i. 在相同的源文档上构建一个RAG系统
documents = SimpleDirectoryReader(input_dir="data/paul_graham").load_data()
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

# ii. 和 iii. 使用`RagEvaluatorPack`进行预测和评估
RagEvaluatorPack = download_llama_pack("RagEvaluatorPack", "./pack")
rag_evaluator = RagEvaluatorPack(
    query_engine=query_engine,
    rag_dataset=rag_dataset,  # 在1A中定义
    show_progress=True,
)

############################################################################
# 注意：如果您有OpenAI API的低级别订阅，比如使用第1层级别 #
# 那么您需要使用不同的batch_size和sleep_time_in_seconds。 #
# 对于第1层级别，似乎效果很好的设置是batch_size=5， #
# 和sleep_time_in_seconds=15（截至2023年12月）。 #
############################################################################

benchmark_df = await rag_evaluator_pack.arun(
    batch_size=20,  # 批量处理要进行的openai api调用数量
    sleep_time_in_seconds=1,  # 在进行api调用之前睡眠的秒数
)

In [ ]:
benchmark_df

rag,base_rag
metrics,
mean_correctness_score,4.238636
mean_relevancy_score,0.977273
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.942281


#### 模板


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.llama_pack import download_llama_pack

documents = SimpleDirectoryReader(  # 可以在这里使用不同的阅读器。
    input_dir=<FILL-IN>  # 应该读取与创建LabelledRagDataset相同的源文件
).load_data()            # 步骤1的数据集。

index = VectorStoreIndex.from_documents( # 或者使用另一个索引
    documents=documents
) 
query_engine = index.as_query_engine()

RagEvaluatorPack = download_llama_pack(
  "RagEvaluatorPack", "./pack"
)
rag_evaluator = RagEvaluatorPack(
    query_engine=query_engine,
    rag_dataset=rag_dataset,  # 在步骤1A中定义
    judge_llm=<FILL-IN>  # 如果您不想使用GPT-4
)
benchmark_df = await rag_evaluator.arun()
benchmark_df

#### [返回顶部](#top) 


<a id='Step3'></a>
## 3. 准备 `card.json` 和 `README.md`

提交数据集时需要提交一些元数据。这些元数据存储在两个不同的文件中，即 `card.json` 和 `README.md`，它们都作为提交包的一部分包含在 `llama-hub` Github 仓库中。为了加快这一步骤并确保一致性，您可以使用 `LlamaDatasetMetadataPack` llamapack。或者，您可以按照下面提供的演示和模板手动完成这一步骤。


<a id='3A'></a>
### 3A. 使用`LlamaDatasetMetadataPack`进行自动生成


#### 演示

这是继续上一节1A中保罗·格雷厄姆（Paul Graham）文章演示示例的内容。


In [ ]:
from llama_index.core.llama_pack import download_llama_pack

LlamaDatasetMetadataPack = download_llama_pack(
    "LlamaDatasetMetadataPack", "./pack"
)

metadata_pack = LlamaDatasetMetadataPack()

dataset_description = (
    "基于Paul Graham的一篇文章的标记RAG数据集，包括查询、参考答案和参考上下文。"
)

# 这将在运行此笔记本的相同目录中创建并保存card.json和README.md文件。
metadata_pack.run(
    name="Paul Graham Essay Dataset",
    description=dataset_description,
    rag_dataset=rag_dataset,
    index=index,
    benchmark_df=benchmark_df,
    baseline_name="llamaindex",
)

In [ ]:
# 如果你想快速查看这两个文件，将take_a_peak设置为True
take_a_peak = False

if take_a_peak:
    import json

    with open("card.json", "r") as f:
        card = json.load(f)

    with open("README.md", "r") as f:
        readme_str = f.read()

    print(card)
    print("\n")
    print(readme_str)

这是一个示例模板，用于演示如何将ipynb文件中的markdown内容翻译成中文。


In [ ]:
from llama_index.core.llama_pack import download_llama_pack

LlamaDatasetMetadataPack = download_llama_pack(
  "LlamaDatasetMetadataPack", "./pack"
)

metadata_pack = LlamaDatasetMetadataPack()
metadata_pack.run(
    name=<填写>,
    description=<填写>,
    rag_dataset=rag_dataset,  # 来自步骤1
    index=index,  # 来自步骤2
    benchmark_df=benchmark_df,  # 来自步骤2
    baseline_name="llamaindex",  # 可选择使用其他名称
    source_urls=<可选填写>,
    code_url=<可选填写>  # 如果您希望提交代码以复制基准结果
)

运行上面的代码后，您可以手动检查`card.json`和`README.md`，并进行必要的编辑，然后提交到`llama-hub` Github存储库。


#### [步骤 4](#Step4), [返回顶部](#top)


<a id='3B'></a>

### 3B. 手动生成


在这部分，我们将演示如何通过使用保罗·格雷厄姆的文章示例，在1A中创建`card.json`和`README.md`文件（如果您选择了1C作为第1步）。

#### `card.json`


#### 演示


```json
{
    "name": "保罗·格雷厄姆的文章",
    "description": "基于保罗·格雷厄姆的一篇文章的标记的RAG数据集，包括查询、参考答案和参考上下文。",
    "numberObservations": 44,
    "containsExamplesByHumans": false,
    "containsExamplesByAI": true,
    "sourceUrls": [
        "http://www.paulgraham.com/articles.html"
    ],
    "baselines": [
        {
            "name": "llamaindex",
            "config": {
                "chunkSize": 1024,
                "llm": "gpt-3.5-turbo",
                "similarityTopK": 2,
                "embedModel": "text-embedding-ada-002"
            },
            "metrics": {
                "contextSimilarity": 0.934,
                "correctness": 4.239,
                "faithfulness": 0.977,
                "relevancy": 0.977
            },
            "codeUrl": "https://github.com/run-llama/llama-hub/blob/main/llama_hub/llama_datasets/paul_graham_essay/llamaindex_baseline.py"
        }
    ]
}
```


#### Template


```
{
    "name": <填写>,
    "description": <填写>,
    "numberObservations": <填写>,
    "containsExamplesByHumans": <填写>,
    "containsExamplesByAI": <填写>,
    "sourceUrls": [
        <填写>,
    ],
    "baselines": [
        {
            "name": <填写>,
            "config": {
                "chunkSize": <填写>,
                "llm": <填写>,
                "similarityTopK": <填写>,
                "embedModel": <填写>
            },
            "metrics": {
                "contextSimilarity": <填写>,
                "correctness": <填写>,
                "faithfulness": <填写>,
                "relevancy": <填写>
            },
            "codeUrl": <可选-填写>
        }
    ]
}
```


#### `README.md`

在这一步中，最低要求是采用下面的模板并填写必要的项目，这意味着将数据集的名称更改为您想要在新提交中使用的名称。


#### 演示

点击[这里](https://raw.githubusercontent.com/run-llama/llama-hub/main/llama_hub/llama_datasets/paul_graham_essay/README.md)查看一个示例`README.md`。


```python
# 模板

这是一个示例模板文件，用于演示如何使用模板。

```


点击[这里](https://raw.githubusercontent.com/run-llama/llama-hub/main/llama_hub/llama_datasets/template_README.md)获取`README.md`的模板。只需复制并粘贴该文件的内容，然后用你选择的新数据集名称替换占位符"[NAME]"和"[NAME-CAMELCASE]"的相应值。例如：
- "{NAME}" = "保罗·格雷厄姆文章数据集"
- "{NAME_CAMELCASE}" = PaulGrahamEssayDataset


#### [返回顶部](#top) 


<a id='Step4'></a>
## 4. 将Pull Request提交到[llama-hub](https://github.com/run-llama/llama-hub)存储库

现在，是时候提交您的新数据集的元数据，并在数据集注册表中创建一个新条目了，这些信息存储在文件`library.json`中（即，请查看[这里](https://github.com/run-llama/llama-hub/blob/main/llama_hub/llama_datasets/library.json)）。

### 4a. 在`llama_hub/llama_datasets`下创建一个新目录，并添加您的`card.json`和`README.md`：
```bash
cd llama-hub  # 进入本地克隆的llama-hub目录
cd llama_hub/llama_datasets
git checkout -b my-new-dataset  # 创建一个新的git分支
mkdir <dataset_name_snake_case>  # 按照其他数据集的约定命名
cd <dataset_name_snake_case>
vim card.json # 使用vim或其他文本编辑器添加card.json的内容
vim README.md # 使用vim或其他文本编辑器添加README.md的内容
```


### 4b. 在 `llama_hub/llama_datasets/library.json` 中创建一个条目

```bash
cd llama_hub/llama_datasets
vim library.json # 使用vim或其他文本编辑器注册你的新数据集
```


```json
  "PaulGrahamEssayDataset": {
    "id": "llama_datasets/paul_graham_essay",
    "author": "nerdai",
    "keywords": ["rag"]
  }
```


```json
  "<填写>": {
    "id": "llama_datasets/<数据集名称蛇形命名>",
    "author": "<填写>",
    "keywords": ["rag"]
  }
```

**注意**: 请使用与4a中相同的`数据集名称蛇形命名`。


### 4c. 将更改`git add`和`commit`，然后推送到您的分支

```bash
git add .
git commit -m "my new dataset submission"
git push origin my-new-dataset
```

完成后，转到[llama-hub](https://github.com/run-llama/llama-hub)的Github页面。您应该会看到从您的分支发起拉取请求的选项。现在就去做吧。


#### [返回顶部](#top) 


<a id='Step5'></a>
## 5. 向 [llama-datasets](https://github.com/run-llama/llama-datasets) 仓库提交拉取请求


在提交过程的最后一步中，您将提交实际的`LabelledRagDataset`（以json格式）以及源数据文件到`llama-datasets` Github存储库。


### 5a. 在`llama_datasets/`下创建一个新目录：

```bash
cd llama-datasets # 进入本地的 llama-datasets 克隆目录
git checkout -b my-new-dataset  # 创建一个新的git分支
mkdir <dataset_name_snake_case>  # 使用步骤4中使用的相同名称。
cd <dataset_name_snake_case>
cp <path-in-local-machine>/rag_dataset.json .  # 添加 rag_dataset.json
mkdir source_files  # 添加所有的源文件
cp -r <path-in-local-machine>/source_files  ./source_files  # 添加所有的源文件
```

**注意**：请使用与步骤4中相同的 `dataset_name_snake_case`。


### 5b. `git add` 和 `commit` 你的更改，然后推送到你的分支

```bash
git add .
git commit -m "我新的数据集提交"
git push origin my-new-dataset
```

完成后，前往 [llama-datasets](https://github.com/run-llama/llama-datasets) 的 Github 页面。你应该会看到从你的分支创建拉取请求的选项。现在就去做吧。


#### [返回顶部](#top)


## 大功告成！

您已经完成了数据集提交流程！🎉🦙恭喜您，感谢您的贡献！
